<a href="https://colab.research.google.com/github/jben-hun/colab_notebooks/blob/master/SteamServerStat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steam server statistics

This notebook uses the [Master Server Query Protocol](https://developer.valvesoftware.com/wiki/Master_Server_Query_Protocol) and [server queries](https://developer.valvesoftware.com/wiki/Server_queries#Protocol) to get a server list and statistics of multiplayer [Source](https://developer.valvesoftware.com/wiki/Source) and [Goldsource](https://developer.valvesoftware.com/wiki/Goldsource) engine mods.
*   This is for mods without an [AppID](https://developer.valvesoftware.com/wiki/Steam_Application_IDs) and own [Steam store page](https://developer.valvesoftware.com/wiki/Steam), for those official mods there are <https://store.steampowered.com/stats/> and <https://steamcharts.com/>
*   Python Steam API package documentation: <https://steam.readthedocs.io/en/stable/api/steam.game_servers.html>
*   Cool standalone server browser: <https://github.com/PredatH0r/SteamServerBrowser>

In [1]:
# interactively display tables
%load_ext google.colab.data_table

%pip -q install steam

In [2]:
import numpy as np
import pandas as pd
from steam import game_servers as gs
from collections import defaultdict


def get_server_list(*app_ids, has_players=True, limit=500):
    servers = []

    for id in app_ids:
        query_string = ""
        query_string += fr"\appid\{id}"
        query_string += fr"\empty\1" if has_players else ""
        sub = list(gs.query_master(query_string, max_servers=limit))
        servers += sub

        print(f"{len(sub)} servers from query: \"{query_string}\"")
        if len(sub) == limit:
            print(f"Warning, query response limit({limit}) reached, the "
                  f"result is possibly truncated, increase the limit")

    print(f"{len(servers)} servers total")

    return servers


def get_server_info(*servers, apps=None):
    infos = []
    for server in servers:
        try:
            info = gs.a2s_info(server)

            info["address"] = ":".join(map(str, server))

            if "app_id" in info:
                info["app"] = (apps[info["app_id"]] if apps is not None
                               else info["app_id"])
            else:
                info["app"] = "unknown"


            infos.append(info)

        except Exception as e:
            print(f"""{info["address"]}: {type(e).__name__}: {e}""")

    return (pd.DataFrame(infos)
            .set_index(["app", "folder"])
            .sort_values(by="players", ascending=False))


def display_player_count(info):
    display(
        info
        .groupby(info.index)
        .agg(players=pd.NamedAgg(column="players", aggfunc="sum"),
             populated_servers=pd.NamedAgg(column="players", aggfunc="count"),
             mean_players_per_server=pd.NamedAgg(column="players",
                                                 aggfunc="mean"),
             game_names=pd.NamedAgg(column="game", aggfunc=",".join))
        .sort_values(by="populated_servers", ascending=False)
    )


def display_server_info(info):
    c = ["address", "name", "app_id", "app", "game_id", "game", "folder",
         "map", "players", "max_players", "bots", "_type", "protocol",
         "server_type", "environment", "vac", "version", "keywords",
         "sourcetv_port", "sourcetv_name"]

    display(info.reset_index().loc[:, c])

In [3]:
# Goldsource mods without a Steam page usually use Half-Life as the base game,
# Source mods use their respective Source SDK Bases
apps = {
    70: "Half-Life",
    215: "Source SDK Base 2006",
    218: "Source SDK Base 2007",
    243750: "Source SDK Base 2013 Multiplayer",
}

In [4]:
servers = get_server_list(*apps.keys())
info = get_server_info(*servers, apps=apps)

52 servers from query: "\appid\70\empty\1"
9 servers from query: "\appid\215\empty\1"
1 servers from query: "\appid\218\empty\1"
9 servers from query: "\appid\243750\empty\1"
71 servers total
192.169.88.50:29035: timeout: timed out
192.169.88.50:29035: RuntimeError: Invalid reponse header - b'D'
18.156.63.218:27015: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: OSError: [Errno 113] No route to host
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
78.46.191.68:27016: timeout: timed out
185.86.78.191:27015: timeout: timed out
185.86.78.191:27015: RuntimeError: Invalid reponse header - b'D'
31.131.249.69:27018: RuntimeError: Invalid reponse header - b'D'
89.223

In [5]:
display_player_count(info)
display_server_info(info)

,players,populated_servers,mean_players_per_server,game_names
"(Half-Life, valve)",143,22,6.500000,"Sev+|GG EDITION,D A W N,H λ L F - L I F E,H..."
"(Half-Life, ag)",13,6,2.166667,"HLCCL,AG FFA,AG TDM,HLCCL,AGT,AG TDM"
"(Source SDK Base 2006, hidden)",4,4,1.000000,"Hidden : Source B4,Hidden : Source B4,Hidden :..."
"(Source SDK Base 2013 Multiplayer, tf2classic)",32,4,8.000000,"Team Fortress 2 Classic,Deathrun Toolkit | 0.4..."
"(Source SDK Base 2006, gmod9)",6,2,3.000000,"GMod 9.0.4,GMod 9.0.4"
"(Half-Life, dpbredux)",3,1,3.000000,Digital Paintball
"(Half-Life, nnk)",1,1,1.000000,Naruto Naiteki Kensei R1
"(Source SDK Base 2006, zombie_master)",2,1,2.000000,Zombie Master 1.2.1
"(Source SDK Base 2007, gesource)",3,1,3.000000,Team Arsenal (MOD)


,address,name,app_id,app,game_id,game,folder,map,players,max_players,bots,_type,protocol,server_type,environment,vac,version,keywords,sourcetv_port,sourcetv_name
0,54.39.130.18:27015,VaultF4.com - Official [US] Standard,243750,Source SDK Base 2013 Multiplayer,243750.0,Team Fortress 2 Classic,tf2classic,cp_coldfront,14,30,0,source,17,d,l,1,5394425,"HLstatsX:CE,alltalk,cp,increased_maxplayers,no...",27020.0,SourceTV Test
1,23.124.59.250:27015,Gamers-Global.com|HLSTATS|SEV+|FASTDL|CUSTOM|1...,70,Half-Life,70.0,Sev+|GG EDITION,valve,7th_path,14,22,4,source,48,d,w,1,1.1.2.2,,NaN,NaN
2,84.22.153.100:27018,Dawn HL Crossfire Only,70,Half-Life,70.0,D A W N,valve,crossfire,14,20,3,source,48,d,w,1,1.1.2.2/Stdio,NaN,NaN,NaN
3,188.213.212.5:27015,|-> -==[24/7]==- HL.Pyro-Zone.com <-|,70,Half-Life,70.0,H λ L F - L I F E,valve,hl_assault,13,32,2,source,48,d,l,1,1.1.2.2/Stdio,NaN,NaN,NaN
4,62.140.250.10:27015,! !--Good_Half-Life_Server--! !,70,Half-Life,70.0,Half-Life,valve,crossfire,13,32,1,source,48,d,w,1,1.1.2.2/Stdio,NaN,NaN,NaN
5,78.152.169.100:27016,[VICTORY.KM.UA] Half-Life DM FFA,70,Half-Life,NaN,Half-Life,valve,closefire,13,32,0,source,48,d,w,0,1.1.2.7/Stdio,NaN,NaN,NaN
6,192.223.30.68:27015,! !--Best_Half-Life_Server--! !,70,Half-Life,70.0,H λ L F - L I F E,valve,crossfire,10,10,0,source,48,d,w,1,1.1.2.2/Stdio,NaN,NaN,NaN
7,84.22.153.100:27015,Dawn Half-Life,70,Half-Life,70.0,D A W N,valve,dead-dust2,9,20,2,source,48,d,w,1,1.1.2.2/Stdio,NaN,NaN,NaN
8,78.47.20.213:27019,⬛█ Death Ventures | Combat / Jailbreak / Death...,243750,Source SDK Base 2013 Multiplayer,243750.0,Deathrun Toolkit | 0.4.1,tf2classic,dr_bank_v12a,9,24,0,source,17,d,l,1,5394425,"deathrun,deathventures,jailbreak,teamfortress2...",NaN,NaN
9,192.169.88.50:29035,*L.T.K*Clan Server 1,70,Half-Life,70.0,Severian's Mod,valve,dutch_wonderfire,7,26,3,source,48,d,w,1,1.1.2.2,NaN,NaN,NaN
